# 4.4 学習の効率化（LoRA）: HuggingFace PEFTを用いたLoRAファインチューニング

このノートブックでは、4.4節の内容に沿って、HuggingFace PEFTライブラリを用いた
LoRA（Low-Rank Adaptation）によるファインチューニングを実装します。

- rinnaの事前学習済み日本語GPT-2モデル (`rinna/japanese-gpt2-medium`) をベースモデルとして使用
- 4.1節で前処理した青空文庫データセットでファインチューニング
- LoRAにより、全パラメータの一部のみを効率的に学習

**前提条件**: `section01_dataset_preprocessing.ipynb` を実行済みで、`data/aozora_preprocessed` が存在すること

In [ ]:
from pathlib import Path

import torch
from datasets import load_from_disk
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoTokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

## データセットの読み込み

4.1節で前処理・保存した青空文庫データセットを読み込み、訓練用と評価用に分割します。

In [ ]:
# 前処理済みデータの読み込み
data_dir = Path("data") / "aozora_preprocessed"
dataset = load_from_disk(str(data_dir))
print(f"読み込んだデータセット: {dataset}")
print(f"サンプル数: {len(dataset)}")

# 訓練/評価に分割（1%を評価用に使用）
EVAL_RATIO = 0.01
split_ds = dataset.train_test_split(test_size=EVAL_RATIO, seed=42)
train_dataset = split_ds["train"]
eval_dataset = split_ds["test"]

print(f"訓練データ: {len(train_dataset)} サンプル")
print(f"評価データ: {len(eval_dataset)} サンプル")

## トークナイザとモデルの読み込み

rinnaが公開している日本語GPT-2 mediumモデルとそのトークナイザを読み込みます。

In [ ]:
MODEL_NAME = "rinna/japanese-gpt2-medium"

# トークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"語彙サイズ: {len(tokenizer)}")
print(f"PAD token: {tokenizer.pad_token}")

# モデルの読み込み
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id

print(f"\nモデル: {MODEL_NAME}")
print(f"レイヤー数: {model.config.n_layer}")
print(f"隠れ層次元: {model.config.n_embd}")
print(f"パラメータ数: {sum(p.numel() for p in model.parameters()):,}")

## データセットのトークン化

テキストデータをトークナイザで数値列に変換します。最大長は512トークンに設定します。

In [ ]:
BLOCK_SIZE = 512
TEXT_COL = "text"


def tokenize_function(examples):
    return tokenizer(
        examples[TEXT_COL],
        truncation=True,
        max_length=BLOCK_SIZE,
    )


tokenized_train = train_dataset.map(
    tokenize_function, batched=True, remove_columns=[TEXT_COL]
)
tokenized_eval = eval_dataset.map(
    tokenize_function, batched=True, remove_columns=[TEXT_COL]
)

print(f"トークン化後の訓練データ: {tokenized_train}")
print(f"トークン化後の評価データ: {tokenized_eval}")

## LoRAの設定と適用

LoRA (Low-Rank Adaptation) では、事前学習済みモデルの重みを凍結し、
各対象層に低ランクの行列ペア（A, B）を追加して学習します。

主要なハイパーパラメータ:
- `r` (ランク): 低ランク行列の次元数。小さいほどパラメータ効率が高い
- `lora_alpha`: スケーリングファクター。実際のスケールは `alpha / r` で決まる
- `lora_dropout`: LoRA層に適用するドロップアウト率
- `target_modules`: LoRAを適用する層の名前。GPT-2では `c_attn`（QKV射影）と `c_proj`（出力射影）が一般的

In [ ]:
# LoRA設定
LORA_RANK = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.1
LORA_TARGET_MODULES = ["c_attn", "c_proj"]

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=LORA_TARGET_MODULES,
)

# LoRAをモデルに適用
model = get_peft_model(model, lora_config)

# 学習可能なパラメータ数を確認
model.print_trainable_parameters()

## 学習の実行

HuggingFace Trainerを使用してLoRAファインチューニングを実行します。
LoRAでは通常のフルファインチューニングよりも高い学習率（1e-4）を使用します。

In [ ]:
# 学習パラメータ
PER_DEVICE_TRAIN_BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 8
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.1
WARMUP_STEPS = 100
NUM_TRAIN_EPOCHS = 3
OUTPUT_DIR = "./models/rinna-gpt2-aozora-lora"

# ログ・評価間隔の計算（1エポックの1/10ごと）
steps_per_epoch = len(tokenized_train) // (
    PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS
)
logging_steps = max(1, steps_per_epoch // 10)
print(f"1エポックあたりのステップ数: {steps_per_epoch}")
print(f"ログ・評価間隔: {logging_steps} steps")

# データコレータ（言語モデル用）
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 学習設定
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_steps=WARMUP_STEPS,
    logging_steps=logging_steps,
    eval_strategy="steps",
    eval_steps=logging_steps,
    save_strategy="steps",
    save_steps=logging_steps,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    dataloader_num_workers=4,
    fp16=torch.cuda.is_available(),
)

# Trainerの作成と学習実行
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    processing_class=tokenizer,
)

trainer.train()

## LoRAアダプターの保存

学習済みのLoRAアダプターを保存します。LoRAではベースモデル全体ではなく、
追加された低ランク行列のみが保存されるため、保存サイズが非常に小さくなります。

In [ ]:
# LoRAアダプターの保存
adapter_dir = Path(OUTPUT_DIR) / "adapter"
model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)
print(f"LoRAアダプター保存先: {adapter_dir}")

## 生成テスト

学習済みモデルを使って、いくつかのプロンプトからテキスト生成を行います。

In [ ]:
# テスト用プロンプト
test_prompts = [
    "吾輩は猫である。名前はまだ無い。",
    "明治時代の",
    "東京の街には",
    "先生は言った。「",
]

model.eval()
device = next(model.parameters()).device

for i, prompt in enumerate(test_prompts, 1):
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=80,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_part = generated_text[len(prompt):]
    print(f"[{i}] プロンプト: {prompt}")
    print(f"    生成結果: {generated_part[:200]}")
    print()

## まとめ

このノートブックでは、以下の手順でLoRAファインチューニングを実装しました:

1. **データ準備**: 4.1節で前処理した青空文庫データを読み込み、訓練/評価に分割
2. **モデル読み込み**: `rinna/japanese-gpt2-medium` をベースモデルとして使用
3. **LoRA適用**: PEFTライブラリで低ランクアダプターを注入（学習パラメータ数を大幅に削減）
4. **学習**: HuggingFace Trainerで効率的にファインチューニング
5. **生成テスト**: 学習済みモデルでテキスト生成を確認

LoRAの利点:
- 学習パラメータ数がフルファインチューニングの1%未満
- メモリ使用量が大幅に削減され、単一GPUでも大規模モデルの学習が可能
- アダプターのみの保存で、ストレージ効率が高い